In [ ]:
import os
import subprocess
import pandas as pd
import csv
import numpy as np

In [ ]:
# You need to create the folder structure below within a local folder, before running this notebook.

#-Main Folder. e.g C:\atoms
 # -repositories  (Folder with the GitHub repositories being analyzed e.g C:\atoms\repositories)
 # -clodeql-cli (Folder with the codeql client. e.g C:\atoms\clodeql-cli)
 # -results (The folder where the results will be saved  e.g C:\atoms\results)
 # -queries (The CodeQL queries being performed. e.g C:\atoms\queries)
 # -codeql-repo (Folder with the codeql repo. e.g C:\atoms\clodeql-repo)
     
#Match the variable below with your main folder
main_folder = ''

folder_repos = main_folder + 'repositories\\'
folder_codeqlcli = main_folder + 'codeql-cli\\'
folder_results = main_folder + 'results\\'
folder_queries = main_folder + 'queries\\'


In [ ]:
def get_last_commit(folder):
    p = subprocess.Popen(['git', 'rev-parse', 'HEAD' ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder)
    stdout,stderr = p.communicate()
    return stdout.decode("utf-8").replace('\n','')
        
    
    

In [ ]:
queries = os.listdir(folder_queries)

In [ ]:
folders = os.listdir(folder_repos)
for f in folders:
    print(folder_repos + f)
    
    p = subprocess.Popen([folder_codeqlcli + 'codeql.exe', 'database', 'create',  f + '-codeql-db', '--language=javascript', '--mode=brutal'], cwd=folder_repos + f, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    stdout,stderr = p.communicate()
    #print(stderr)
    p = subprocess.Popen([folder_codeqlcli + 'codeql.exe', 'database', 'upgrade',  f + '-codeql-db'], cwd=folder_repos + f, stdout=subprocess.PIPE, stderr=subprocess.STDOUT )
    stdout,stderr = p.communicate()
    
    p.wait()
    

    

In [ ]:
def executeQuery(query):
    #global df_atoms
    df_atoms = pd.DataFrame(columns=['full_name','atom','location','url'])
    atom_name = query.replace('.ql','')
    atom_name = atom_name.replace('Select','')
    for f in folders:
        path =  folder_repos + f
        print('executing query ' + atom_name + ' on '+ folder_repos + f)
        p0 = subprocess.Popen(['rm', f + '.bqrs'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder_repos + f)
        stdout,stderr = p0.communicate()
        
        p = subprocess.Popen([folder_codeqlcli + 'codeql.exe', 'query', 'run', '--output',  path + f + '.bqrs', '-d', f + '-codeql-db', folder_queries + query ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder_repos + f)
        stdout,stderr = p.communicate()
        p2 = subprocess.Popen([folder_codeqlcli + 'codeql.exe', 'bqrs', 'decode', path + f + '.bqrs', '--no-titles', '--format=csv', '--output', f + '.csv' ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder_repos + f)
        stdout,stderr = p2.communicate()
        try:
            df_temp = pd.read_csv(folder_repos + f + '\\' + f + '.csv',delimiter=',',header=None)
        except:
            print('atom of confusion not found in this repository' )
        else:
            
            df_temp.columns = ['atom','location','url']
            df_temp.insert(0, 'atom_name', atom_name)
            full_name = f.replace('_','/')
            
            df_temp['url'] = df_temp['url'].str.replace(folder_repos + f, 'https://github.com/' + full_name + '/blob/' + get_last_commit(folder_repos + f) , regex=False)
            
            df_temp.insert(1, "full_name", full_name) 
            df_atoms = df_atoms.append(df_temp)
        print(stdout)
        
    print('we found ' + str(len(df_atoms)) + ' atoms')     
    return df_atoms

In [ ]:
for query in queries:
    df = executeQuery(query)
    csv_file = query.replace('.ql','.csv')
    csv_file = csv_file.replace('Select','')
    df['url'] = df['url'] + '#L' + df['location'].map(str)
    df.to_csv(folder_results + csv_file ,index=None,quotechar='"',quoting=csv.QUOTE_ALL)
    